# Training a Simple CNN

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from data.image_dataset import ImageDataset
from training.cnn import ConvNet
from training.train import *
from training.eval import *
from data.convert_labels import *
import matplotlib.pyplot as plt

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
train_dir_path = 'data/real/01_20_160_120/train'
train_annotations_file_path = f'{train_dir_path}/01_20_160_120_train.csv'
test_dir_path = 'data/real/01_20_160_120/test'
test_annotations_file_path = f'{test_dir_path}/01_20_160_120_test.csv'

# Can override arguments specifying class boundaries
target_transform = lambda target: convert_labels(target)

train_val_dataset = ImageDataset(
  annotations_file=train_annotations_file_path,
  img_dir=train_dir_path,
  transform=None,
  target_transform=target_transform
)
test_dataset = ImageDataset(
  annotations_file=test_annotations_file_path,
  img_dir=test_dir_path,
  transform=None,
  target_transform=target_transform
)

generator = torch.Generator().manual_seed(0)
train_dataset, val_dataset = random_split(train_val_dataset, [0.9, 0.1], generator=generator)

FileNotFoundError: [Errno 2] No such file or directory: 'data/real/01_20_160_120/train/01_20_160_train.csv'

In [ ]:
# Create Dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# train_features, train_labels = next(iter(train_loader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# print(np.unique(train_labels))

Feature batch shape: torch.Size([64, 1, 120, 160])
Labels batch shape: torch.Size([64])
[ 0  1  2  3  5  6  7  8  9 10 11 13 14 15]


In [ ]:
model = ConvNet(input_dims=(160, 120))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

results = train(model, criterion, optimizer, train_loader, val_loader, epochs=10)

In [ ]:
iter_train_loss_list, train_acc_list = results
plt.plot(iter_train_loss_list)
plt.xlabel('Iteration Number')
plt.ylabel('Loss Value')
plt.title('Loss History')
plt.show()

plt.plot(train_acc_list)
plt.xlabel('Epoch number')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.show()

In [ ]:
test_acc = evaluate_acc(model, test_loader)
print(test_acc)